In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
!pip install tmdbsimple
import tmdbsimple as tmdb
import os, time,json

# Part 1 

# - Importing the Data

In [2]:
basics = pd.read_csv('Data/title_basics_cleaned.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [3]:
ratings_url='Data/title_ratings_cleaned.csv.gz'
ratings = pd.read_csv(ratings_url)

In [4]:
akas_url='Data/title_akas_cleaned.csv.gz'
df_reader = pd.read_csv(akas_url)
df_reader

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
...,...,...,...,...,...,...,...,...
1430934,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0.0
1430935,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0.0
1430936,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0.0
1430937,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0.0


In [8]:
df_reader.replace({'\\N':np.nan},inplace=True)
df_reader= df_reader[(df_reader['region'] == 'US')]
df_reader

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,6,Carmencita,US,NaN,imdbDisplay,NaN,0.0
1,tt0000002,7,The Clown and His Dogs,US,NaN,NaN,literal English title,0.0
2,tt0000005,10,Blacksmith Scene,US,NaN,imdbDisplay,NaN,0.0
3,tt0000005,1,Blacksmithing Scene,US,NaN,alternative,NaN,0.0
4,tt0000005,6,Blacksmith Scene #1,US,NaN,alternative,NaN,0.0
...,...,...,...,...,...,...,...,...
1430934,tt9916560,1,March of Dimes Presents: Once Upon a Dime,US,NaN,imdbDisplay,NaN,0.0
1430935,tt9916620,1,The Copeland Case,US,NaN,imdbDisplay,NaN,0.0
1430936,tt9916702,1,Loving London: The Playground,US,NaN,NaN,NaN,0.0
1430937,tt9916756,1,Pretty Pretty Black Girl,US,NaN,imdbDisplay,NaN,0.0


In [12]:
for temp_df in df_reader:
        #### COMBINED WORKFLOW FROM ABOVE
    ## Replace "\N" with np.nan
    df_reader.replace({'\\N':np.nan},inplace=True)
    df_reader = df_reader[(df_reader['region'] == 'US')]


# Filtering/ Cleaning
1. Title Basics

In [14]:
basics.replace({'\\N':np.nan},inplace=True)

In [15]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


In [17]:
basics = basics.dropna(subset=["titleType"])

In [18]:
basics = basics.dropna(subset=["titleType"])

In [19]:
basics = basics.loc[(basics['titleType']=='movies')] 
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [20]:
basics = basics[(basics['titleType']>=2000)&(basics['titleType']<2022)]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [21]:
# Exclude movies that are included in the documentary category.
is_documentary = basics['titleType'].str.contains('titleType',case=False)
basics_df = basics[~is_documentary]


In [22]:
# Filter the basics table down to only include the US by using the filter akas dataframe
keepers =basics['titleType'].isin(basics['titleType'])
keepers

Series([], Name: titleType, dtype: bool)

In [23]:
basics = basics[keepers]
basics

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres


In [24]:
# example making new folder with os
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")

['.ipynb_checkpoints',
 'final_tmdbdata2000.csv.gz',
 'final_tmdbdata2001.csv.gz',
 'title_akas_chunk_001.csv.gz',
 'title_akas_cleaned.csv.gz',
 'title_basics.csv.gz',
 'title_basics_cleaned.csv.gz',
 'title_ratings_cleaned.csv.gz',
 'tmdb_apiresults2000.json',
 'tmdb_apiresults2001.json']

In [25]:
basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)

In [26]:
basics = pd.read_csv("Data/title_basics.csv.gz", low_memory = False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
